In [1]:
from bayesian_network import BayesianNetwork 
from bayesian_node import BayesianNode
import numpy as np

In [2]:
net = BayesianNetwork(values=["True", "False"]) 
n1 = BayesianNode("A")
net.add_node(n1)
n2 = BayesianNode("B")
net.add_node(n2)
net.add_edge("A", ["B"])

In [3]:
print(net)
net.check_valid()
'''
WAKE UP ANGRY:
noisy neighbours
call from callcenter
'''

Nodes: dict_keys(['A', 'B']), Edges: {'A': ['B']}


KeyError: 'B'